python 3.9.4

In [20]:
import pandas as pd
from datetime import date
import re
import csv

In [21]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [22]:
class Formatar():
    def __init__(self, df, colunas_data):
        self.df = df
        self.colunas_data = colunas_data
        self.coluna_metadados = 'metadados'

    def info_adicional(self, texto):
        info_adicional = [ 
        'ENEM'
        'EAD'
        'SISU'
        'Cancelado'
        ]
        texto_final = ''
        for info in info_adicional:
            if info in texto:
                texto_final += f'''{info}\n''' 
        return f'''{self.coluna}: {texto_final}''' if texto_final != '' else ''

    def encontrar_datas(self, texto):
        r = re.compile(r'\d{2}/\d{2}/\d{4}')
        s =  re.compile(r'(\d{2}/\d{2}[^/0-9])')
        r = str(r.findall(texto))[2:-2]
        s = str(s.findall(texto))[2:-3]
        if r != '':
            self.dia = r[:2]
            self.mes = r[3:5]
            self.ano = r[6:]
            self.data = f'''{self.ano}-{self.mes}-{self.dia}'''
        elif s != '':
            self.dia = s[:2]
            self.mes = s[3:5]
            self.data = f'''-{self.mes}-{self.dia}'''
        else:
            self.data = ''
            self.metadados = self.info_adicional(texto)
    
    def adicionar_ano(self, texto):
        r = re.compile(r'\d{2}/\d{2}/\d{4}')
        r = str(r.findall(texto))[2:-2]
        if r != '':
            self.proximo_dia = r[:2]
            self.proximo_mes = r[3:5]
            self.proximo_ano = r[6:]
        else:
            self.proximo_dia = ''
            self.proximo_mes = ''
            self.proximo_ano = ''

    def tratar_datas(self):
        for index, row in self.df.iterrows():
            for i in range(len(self.colunas_data)):
                self.coluna = self.colunas_data[i] 
                try:
                    self.proxima_coluna = self.colunas_data[i+1] #caso a data não tenha o ano, pegamos da próxima data (ex: incrições de 20/10 a 20/11/2021)
                    self.adicionar_ano(row[self.proxima_coluna])
                except:
                    self.proxima_coluna = ''
                self.metadados = ''
                self.encontrar_datas(row[self.coluna])
                if self.data != '':
                    if str(self.data)[0] == '-':
                        self.data = str(self.proximo_ano) + str(self.data)
                self.df.loc[index, self.coluna_metadados] += self.metadados
                self.df.loc[index, self.coluna] = self.data

            

In [23]:
df = pd.read_csv('ranking.csv', sep = ',', encoding = 'utf-8', dtype = str)
df.fillna('', inplace = True)

colunas = [
    'Inicio incrição', 'Fim inscrição', 'Início isenção ', 'Fim isenção',
    'Primiera fase', 'Primeira fase (segundo dia)', 'Segunda fase',
    'Segunda fase (segundo dia)', 'Resultado'
    ]



In [24]:
df_teste = Formatar(df, colunas)
df_teste.tratar_datas()

In [25]:
df_teste.tratar_datas()

In [26]:
df_teste.df

,Ranking,Instituição,Pública ou Privada,Estado,Inicio incrição,Fim inscrição,Início isenção,Fim isenção,Primiera fase,Primeira fase (segundo dia),Segunda fase,Segunda fase (segundo dia),Resultado,link site,metadados,Posição em Ensino,Nota em Ensino,Posição em Pesquisa,Nota em Pesquisa,Posição em Mercado,Nota em Mercado,Posição em Inovação,Nota em Inovação,Posição em Internacionalização,Nota em Internacionalização,Nota,Extrator
0,1,USP,Estadual,SP,,,,,,,,,,https://www.fuvest.br/wp-content/uploads/fuves...,,,,,,,,,,,,,
1,2,UNICAMP,Estadual,SP,,,,,,,,,,https://www.comvest.unicamp.br/publicado-o-edi...,,,,,,,,,,,,,
2,3,UFRJ,Federal,RJ,,,,,,,,,,,,,,,,,,,,,,,
3,4,UFMG,Federal,MG,,,,,,,,,,,,,,,,,,,,,,,
4,5,UFRGS,Federal,RS,,,,,,,,,,,,,,,,,,,,,,,
5,6,UNESP,Estadual,SP,,,,,,,,,,https://drive.google.com/file/d/1W3Ntu8ZTOTidQ...,,,,,,,,,,,,,
6,7,UFSC,Federal,SC,,,,,,,,,,https://vestibular2022.ufsc.br/agenda2/,,,,,,,,,,,,,
7,8,UFPR,Federal,PR,,,,,,,,,,https://www.ufpr.br/portalufpr/noticias/vestib...,,,,,,,,,,,,,
8,9,UNB,Federal,DF,,,,,,,,,,,,,,,,,,,,,,,
9,10,UFPE,Federal,PE,,,,,,,,,,,,,,,,,,,,,,,


In [27]:
def order_ranking(df):
    df['Ranking'] = df['Ranking'].astype(int)
    df = df.sort_values(by = 'Ranking')
    return display(df)

In [28]:
df_teste.df.to_csv('ranking_parcial(formatado).csv', index = False, sep = ',', encoding= 'utf-8')